In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.7173, 0.5319, 0.1682],
        [0.4063, 0.2086, 0.9212],
        [0.0931, 0.0703, 0.2424],
        [0.0771, 0.2875, 0.3451],
        [0.9343, 0.8505, 0.3387]])


In [7]:
import wandb
!wandb login 6efb86bf60c1e6753f7cd318f46f09fb910b86a8

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin/.netrc


In [8]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

### Setup the `device` variable.

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Download and Prepare Dataset

In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
    transforms.ColorJitter()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

c:\d\python\deep_learning\env\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# Define Model Architecture

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        ## Conv 1st Block
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [13]:
def train(model, device, train_loader, optimizer, epoch, steps_per_epoch=20):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data.to(device), target.to(device)
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = F.nll_loss(output, target)

    scores, predictions = torch.max(output.data, 1)
    train_total += target.size(0)
    train_correct += int(sum(predictions == target))
            
    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()
    
    # Update the neural network weights
    optimizer.step()

  acc = round((train_correct / train_total) * 100, 2)
  print('Epoch [{}], Loss: {}, Accuracy: {}'.format(epoch, loss.item(), acc), end='')
  wandb.log({'Train Loss': loss.item(), 'Train Accuracy': acc})

In [14]:
def test(model, device, test_loader, classes):
  # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.eval()
  test_loss = 0
  test_total = 0
  test_correct = 0

  example_images = []
  with torch.no_grad():
      for data, target in test_loader:
          # Load the input features and labels from the test dataset
          data, target = data.to(device), target.to(device)
          
          # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
          output = model(data)
          
          # Compute the loss sum up batch loss
          test_loss += F.nll_loss(output, target, reduction='sum').item()
          
          scores, predictions = torch.max(output.data, 1)
          test_total += target.size(0)
          test_correct += int(sum(predictions == target))
          
          # WandB – Log images in your test dataset automatically, along with predicted and true labels by passing pytorch tensors with image data into wandb.Image
          # example_images.append(wandb.Image(
          #     data[0], caption="Pred: {} Truth: {}".format(classes[pred[0].item()], classes[target[0]])))
  acc = round((test_correct / test_total) * 100, 2)
  print(' Test_loss: {}, Test_accuracy: {}'.format(test_loss/test_total, acc))
  wandb.log({'Test Loss': test_loss, 'Test Accuracy': acc})

In [15]:
net = Net().to(device)
print(net)

optimizer = optim.Adam(net.parameters())

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [16]:
wandb.init(project='common-ml-errors')
wandb.watch(net, log='all')

for epoch in range(10):
  train(net, device, trainloader, optimizer, epoch)
  test(net, device, testloader, classes)

print('Finished Training')
wandb.finish()

c:\d\python\deep_learning\env\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [0], Loss: 0.7026039361953735, Accuracy: 69.35 Test_loss: 0.5836685862064361, Test_accuracy: 84.11
Epoch [1], Loss: 0.32897791266441345, Accuracy: 87.28 Test_loss: 0.34520052094459536, Test_accuracy: 90.17
Epoch [2], Loss: 0.2442597895860672, Accuracy: 90.85 Test_loss: 0.21794201847910882, Test_accuracy: 93.71
Epoch [3], Loss: 0.29293766617774963, Accuracy: 93.68 Test_loss: 0.1825956507563591, Test_accuracy: 94.58
Epoch [4], Loss: 0.06246434524655342, Accuracy: 94.87 Test_loss: 0.1415964467987418, Test_accuracy: 95.88
Epoch [5], Loss: 0.1287214457988739, Accuracy: 95.01 Test_loss: 0.15999461981803179, Test_accuracy: 95.41
Epoch [6], Loss: 0.14805512130260468, Accuracy: 95.98 Test_loss: 0.1272623852841556, Test_accuracy: 96.02
Epoch [7], Loss: 0.15790097415447235, Accuracy: 95.54 Test_loss: 0.11555568621605634, Test_accuracy: 96.63
Epoch [8], Loss: 0.04254532605409622, Accuracy: 96.13 Test_loss: 0.10558980406969785, Test_accuracy: 96.83
Epoch [9], Loss: 0.05276499688625336, Accura

Train Loss,0.05276
Train Accuracy,95.91
_runtime,30
_timestamp,1630229944
_step,19
Test Loss,946.26822
Test Accuracy,97.14


Train Loss,█▄▃▄▁▂▂▂▁▁
Train Accuracy,▁▆▇▇██████
_runtime,▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▆▇▇▇█
_timestamp,▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▆▇▇▇█
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
Test Loss,█▅▃▂▂▂▁▁▁▁
Test Accuracy,▁▄▆▇▇▇▇███
